In [2]:
!pip install mxnet                      > /dev/null
!pip install gluonnlp pandas tqdm       > /dev/null
!pip install sentencepiece              > /dev/null
!pip install transformers==3            > /dev/null
!pip install torch         

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master    > /dev/null

  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-w3qmo13o


In [4]:
import random
import numpy as np
import pandas as pd
import re

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp

from tqdm.notebook import tqdm

In [5]:
seed = 2021

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)  # Set random seed for all devices
torch.cuda.manual_seed_all(seed)  # Set random seed for all GPUs. If there is no GPU, it will be ignored by de

In [6]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [7]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
# GPU 사용 시
device = torch.device("cuda:0")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [25]:
df = pd.read_csv('데이터_추가2_전처리(불용어추가).csv')
df = df[['new_title3', 'topic_idx']]
df.dropna(inplace=True)

train_df = df.sample(frac=0.8, random_state=2021)
val_df = df.drop(train_df.index)

train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

train_df.shape, val_df.shape

((42719, 2), (10680, 2))

In [26]:
train_df.to_csv('title_news_train.tsv', sep='\t', index=False)
val_df.to_csv('title_news_val.tsv', sep='\t', index=False)

In [27]:
dataset_train = nlp.data.TSVDataset('title_news_train.tsv', field_indices=[0,1], num_discard_samples=1)
dataset_val = nlp.data.TSVDataset('title_news_val.tsv', field_indices=[0,1], num_discard_samples=1)

In [28]:
dataset_train[0], dataset_val[-1]

([' 경남 대학가 등록금 반환 운동 열풍 동참 잇따르다', '2'],
 [' 박근혜 대통령 내달 방한 장관 이집트 대통령 정상 회의', '6'])

In [29]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [30]:
max_len = 64               
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3               
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [31]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [32]:
data_train = BERTDataset(dataset=dataset_train, sent_idx=0, label_idx=1, bert_tokenizer=tok, max_len=max_len, pad=True, pair=False)
data_val = BERTDataset(dataset=dataset_val, sent_idx=0, label_idx=1, bert_tokenizer=tok, max_len=max_len, pad=True, pair=False)

In [33]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [34]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [35]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [36]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [37]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [38]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [39]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [40]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [41]:
for e in range(num_epochs):
    train_acc = 0.0
    val_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train accuracy {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(val_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        val_acc += calc_accuracy(out, label)
    print("epoch {} validation accuracy {}".format(e+1, val_acc / (batch_id+1)))
    print()

  0%|          | 0/668 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.0028486251831055 train acc 0.125
epoch 1 batch id 201 loss 0.5621029138565063 train acc 0.5795242537313433
epoch 1 batch id 401 loss 0.5225285887718201 train acc 0.7168017456359103
epoch 1 batch id 601 loss 0.4299778342247009 train acc 0.7658589850249584
epoch 1 train accuracy 0.7772690506084604


  0%|          | 0/167 [00:00<?, ?it/s]

epoch 1 validation accuracy 0.8842092600513258



...

epoch 2 validation accuracy 0.8927769461077845



  0%|          | 0/668 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.11867138743400574 train acc 0.96875
epoch 3 batch id 201 loss 0.30924490094184875 train acc 0.9300373134328358
epoch 3 batch id 401 loss 0.3027347922325134 train acc 0.9358634663341646
epoch 3 batch id 601 loss 0.13329294323921204 train acc 0.9393198835274542
epoch 3 train accuracy 0.9394882110778443


  0%|          | 0/167 [00:00<?, ?it/s]

epoch 3 validation accuracy 0.8952764114627888

